<div style="background:#5D6D7E;padding:20px;color:#ffffff;margin-top:10px;">

# NLP - Proyecto Final: Generador de Frases por Tema

## Profesora: Lisibonny Beato  
</div>


## Objetivo del Proyecto

El objetivo de este proyecto es construir un generador inteligente de frases motivacionales que, dado un tema específico, sea capaz de generar citas originales, coherentes e inspiradoras alineadas con dicho tema.

Para lograr esto, se utiliza un conjunto de datos que contiene citas reales asociadas a categorías clave como *leadership*, *motivation*, *success*, *failure* y *risk*. Estas frases se clasifican semánticamente utilizando modelos de clasificación **zero-shot** como `MoritzLaurer/deberta-v3-large-zeroshot-v2.0`, y se filtran según su significado y coherencia temática.

Para la generación de nuevas frases, se emplea el modelo de lenguaje **Mixtral 8x7B Instruct**, uno de los más avanzados actualmente disponibles. Este modelo permite producir frases profundas y emocionalmente impactantes a partir de ejemplos y prompts cuidadosamente diseñados.

El sistema combina procesamiento previo, clasificación temática, filtrado por sentido y generación contextual, con el objetivo de crear un banco de frases motivacionales auténticas, diversas y aplicables a distintos contextos personales o profesionales.


##  Librerías Utilizadas

Este proyecto combina herramientas del ecosistema de Python para procesamiento de texto, aprendizaje automático y generación de lenguaje natural. A continuación se describen las librerías y módulos principales importados:

- **`pandas`, `numpy`**: Manipulación de datos estructurados y cálculos numéricos.
- **`matplotlib.pyplot`**: Visualización gráfica de métricas y resultados.
- **`CountVectorizer` (sklearn)**: Conversión de texto a vectores de frecuencia (modelo Bag-of-Words).
- **`train_test_split` (sklearn)**: División del conjunto de datos en entrenamiento y prueba.
- **`sklearn.metrics`**: Evaluación de modelos mediante métricas como:
  - `accuracy_score`: exactitud global
  - `confusion_matrix` y `ConfusionMatrixDisplay`: matriz de confusión y su visualización
  - `classification_report`: resumen con precisión, recall y F1-score por clase
  - `precision_score`, `recall_score`, `f1_score`: métricas individuales
  - `roc_auc_score`, `roc_curve`: evaluación de modelos binarios con curvas ROC
- **`re`, `random`, `gc`**: Utilidades de expresiones regulares, aleatoriedad y limpieza de memoria.
- **`torch`**: Uso de la GPU (CUDA) con PyTorch para acelerar modelos de deep learning.
- **`tqdm`**: Progreso visual en bucles largos.

###  Hugging Face Transformers
- **`pipeline`**: API de alto nivel para tareas como clasificación o generación de texto.
- **`AutoTokenizer`, `AutoModelForCausalLM`**: Utilizados para cargar modelos preentrenados de lenguaje, como `Mixtral` o `Nous Hermes`.

###  Autenticación
- **`huggingface_hub.login`**: Permite autenticarse con Hugging Face para acceder a modelos privados o acelerados vía token personal.



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, confusion_matrix, ConfusionMatrixDisplay,
    classification_report, precision_score, recall_score,
    f1_score, roc_auc_score, roc_curve
)
import re
import random
import torch
import gc
from tqdm import tqdm
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

from huggingface_hub import login
login("TOKEN")

###  Instalación de dependencias y recursos NLTK

Antes de ejecutar el sistema, asegúrate de instalar las librerías necesarias y descargar los recursos de NLTK:

In [11]:
!pip install -U bitsandbytes
!pip install -U transformers accelerate

###  Carga del Dataset

Se utiliza un conjunto de datos llamado `quotes-wisdom.csv`, el cual contiene frases motivacionales clasificadas por tema. A continuación, se carga en un DataFrame de pandas:


In [12]:
df = pd.read_csv("quotes-wisdom.csv")

###  Exploración de columnas del dataset

Se visualizan las columnas disponibles en el archivo `quotes-wisdom.csv` para conocer la estructura del conjunto de datos. Esto permite identificar qué campos pueden ser útiles para el análisis y la generación de frases.


In [13]:
print("Columnas del dataset original:")
print(df.columns.tolist())

Columnas del dataset original:
['quote', 'author', 'theme/tag', 'source', 'position', 'region', 'decade', 'gender']


###  Exploración de temas disponibles

Se identifican los temas únicos presentes en la columna `theme/tag` del dataset. Esto permite conocer las categorías originales de las frases, las cuales pueden utilizarse para agrupar, filtrar o seleccionar ejemplos de entrenamiento para el generador de frases.


In [14]:
temas_unicos = df["theme/tag"].dropna().unique()
print("Temas disponibles:")
for i, t in enumerate(temas_unicos, 1):
    print(f"{i}. {t}")

Temas disponibles:
1. leadership
2. motivation
3. success
4. failure
5. risk


###  Limpieza inicial del dataset

Se crea una copia del dataset original para trabajar de forma segura (`df_base`). Luego, se eliminan aquellas filas que no contienen una frase (`quote`), lo que asegura que solo se conserven entradas válidas para el proceso de análisis y generación. Tras esta limpieza, se muestra la cantidad de frases válidas restantes.


In [15]:
df_base = df.copy()

df_base = df_base.dropna(subset=["quote"])
print(f"Frases válidas tras eliminar nulos: {len(df_base)}")

Frases válidas tras eliminar nulos: 7869


###  Función de filtrado de frases irrelevantes

Se define una función `filtrar_quotes_basura()` que recibe un DataFrame con frases y elimina aquellas que no son útiles para generar citas motivacionales. Entre los elementos que se descartan se encuentran:

- Frases que mencionan fuentes periodísticas, redes sociales, entrevistas, blogs, documentales, podcasts o referencias bibliográficas.
- Frases que contienen URLs o nombres de medios como CNN, BBC, Forbes, etc.
- Frases que incluyen fechas explícitas (años, meses o días), patrones como `(2023)` o formatos como `12/05/2021`.
- Frases que inician con expresiones como “as quoted” o que contienen texto entre corchetes.

El objetivo es mantener únicamente frases limpias, atemporales y con sentido general, listas para ser clasificadas y usadas como ejemplos de entrenamiento o inspiración.


In [16]:
def filtrar_quotes_basura(df: pd.DataFrame, col: str = "quote") -> pd.DataFrame:
    frases = df[col].astype(str).str.lower()

    filtros_keywords = [
        "retrieved", "interview", "video", "episode", "article", "quoted", "speech", "press release",
        "podcast", "q&a", "transcript", "watch", "full episode", "hosted by", "said to", "reporter", "chapter",
        "isbn", "page", "source", "source:", "http", "https", "youtube", "nyt", "bloomberg", "wired", "yahoo",
        "cnn", "bbc", "zdnet", "fortune", "reuters", "independent", "documentary", "blog", "vulture", "ted talk",
        "space.com", "science magazine", "science journal", "tweet", "response", "cited in", "twitter", "macrumors",
        "population", "number of children", "off the record", "screenshot", "statement", "email", "media outlet",
        "press", "news", "magazine", "forbes", "remarks at", "at the", "address at", "worldwide developers conference",
        "presentation on", "buffetcup", "conference", "developer event", "event", "convention center", "summit"
    ]

    mask_excluir = frases.str.contains('|'.join(filtros_keywords), na=False)

    regex_fechas = [
        r"\b\d{1,2} (?:january|february|march|april|may|june|july|august|september|october|november|december) \d{4}\b",
        r"\b(?:january|february|march|april|may|june|july|august|september|october|november|december) \d{4}\b",
        r"\b\d{4}\b",
        r"\b\d{1,2} (?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[^\w]",
        r"\(\d{4}\)", r"\[\d{4}\]",
        r"\b\d{1,2}[/-]\d{1,2}[/-]\d{2,4}\b"
    ]

    for patron in regex_fechas:
        mask_excluir |= frases.str.contains(patron, na=False, regex=True)

    mask_excluir |= frases.str.startswith("as quoted", na=False)
    mask_excluir |= frases.str.contains(r"\[.*?\]", na=False)
    mask_excluir |= frases.str.contains(r"^\s*from ", na=False)

    return df[~mask_excluir].copy()

df_limpio = filtrar_quotes_basura(df_base)

print(f"Antes (base): {len(df_base)} frases")
print(f"Después (limpio): {len(df_limpio)} frases")


Antes (base): 7869 frases
Después (limpio): 4197 frases


###  Función para recuperar metadatos

La función `recuperar_metadata()` toma dos DataFrames:

- `df_original`: el dataset completo con columnas como `"quote"`, `"author"` y `"theme/tag"`.
- `df_limpio`: el dataset filtrado que contiene solo frases válidas.

La función realiza un `merge` para devolver al DataFrame limpio sus columnas originales de autor y tema, utilizando como clave la columna `"quote"`.

Esto permite mantener el contexto original de cada frase incluso después del proceso de limpieza.


In [17]:
def recuperar_metadata(df_original: pd.DataFrame, df_limpio: pd.DataFrame) -> pd.DataFrame:
    if not all(c in df_original.columns for c in ["quote", "author", "theme/tag"]):
        raise ValueError("Faltan columnas requeridas")

    return pd.merge(
        df_limpio[["quote"]],
        df_original[["quote", "author", "theme/tag"]],
        on="quote",
        how="left"
    )

###  Visualización de frases limpias y columnas finales

Una vez recuperados los metadatos con la función `recuperar_metadata()`, se crea el DataFrame `df_final`, que contiene las frases limpias junto con su autor y tema original.

Se imprime:

- La lista de columnas disponibles en `df_final`.
- Las primeras frases procesadas correctamente para verificar el resultado de la limpieza.


In [18]:
df_final = recuperar_metadata(df, df_limpio)

print("Columnas finales:", df_final.columns.tolist())
print("Primeras frases limpias:")
print(df_final.head())

Columnas finales: ['quote', 'author', 'theme/tag']
Primeras frases limpias:
                                               quote          author  \
0  It’s only after you’ve stepped outside your co...  Roy T. Bennett   
1  Success is not how high you have climbed, but ...  Roy T. Bennett   
2  Be grateful for what you already have while yo...  Roy T. Bennett   
3  Let the improvement of yourself keep you so bu...  Roy T. Bennett   
4  Listen with curiosity. Speak with honesty. Act...  Roy T. Bennett   

    theme/tag  
0  leadership  
1  leadership  
2  leadership  
3  leadership  
4  leadership  


###  Configuración de modelos para generación y clasificación

Se utilizan dos modelos principales en esta fase del proyecto:

#### 🔹 Generador de frases (Mixtral 8x7B Instruct)
Se carga el modelo `mistralai/Mixtral-8x7B-Instruct-v0.1`, optimizado para instrucciones. Se utiliza `.generate()` en lugar del `pipeline` para un control más preciso sobre los parámetros de generación como:

- `temperature`: controla la creatividad del texto.
- `num_return_sequences`: cuántas frases se generan por prompt.
- `max_new_tokens`: longitud máxima de cada frase generada.

> También se incluye, aunque comentado, el modelo alternativo `Nous Hermes 2 - Mistral 7B DPO`.

#### 🔹 Clasificadores DeBERTa (Zero-shot)
Se emplea `MoritzLaurer/deberta-v3-large-zeroshot-v2.0` para dos tareas:

- **Clasificador de sentido** (`sentido_classifier`): Determina si una frase es significativa.
- **Clasificador de tema** (`tema_classifier`): Clasifica frases en los temas: `leadership`, `motivation`, `success`, `failure`, `risk`.

Ambos clasificadores usan el pipeline de Hugging Face y se ejecutan con GPU si está disponible (`cuda:0`).


In [19]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

# ✅ Detectar GPU
device = 0 if torch.cuda.is_available() else -1
print(f"📦 Dispositivo usado: {'cuda:0' if device == 0 else 'CPU'}")

# ⚙️ Configuración opcional para L4 (no necesaria en A100, pero se deja comentada)
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.float16
# )

# ✅ Modelo Nous Hermes 2 (comentado por defecto)
# model_id = "NousResearch/Nous-Hermes-2-Mistral-7B-DPO"
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     device_map="auto",
#     trust_remote_code=True,
#     quantization_config=bnb_config,
#     torch_dtype=torch.float16
# )

#  Modelo Mixtral 8x7B Instruct
model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16
)

#  Función para generar frases usando .generate()
def generar_frases(prompt, num_return_sequences=10, max_new_tokens=50, temperature=0.85):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=temperature,
        num_return_sequences=num_return_sequences,
        pad_token_id=tokenizer.eos_token_id
    )
    frases = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return [f.replace(prompt, "").strip() for f in frases]

#  Clasificadores DeBERTa
from transformers import pipeline

sentido_classifier = pipeline(
    "zero-shot-classification",
    model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0",
    device=device,
    batch_size=8
)

tema_classifier = pipeline(
    "zero-shot-classification",
    model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0",
    device=device,
    batch_size=8
)

#  Etiquetas
labels_sentido = ["meaningful", "not meaningful"]
labels_temas = ["leadership", "motivation", "success", "failure", "risk"]


📦 Dispositivo usado: cuda:0


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

...

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/970 [00:00<?, ?B/s]

Device set to use cuda:0
Device set to use cuda:0


### Clasificación de Frases por Sentido

Se define un umbral (`UMBRAL_SENTIDO = 0.70`) que determina si una frase tiene "sentido" o no. Luego, se aplica un clasificador `zero-shot` usando el modelo DeBERTa para evaluar, por lotes, todas las frases del conjunto limpio (`df_limpio`).

Solo se consideran válidas aquellas frases clasificadas como `"meaningful"` y que superen el umbral establecido. Las frases aceptadas se almacenan en la lista `frases_validas`, incluyendo su puntuación de sentido (`sentido_score`).

Este proceso permite filtrar el dataset inicial para trabajar únicamente con frases que sean comprensibles y relevantes.


In [20]:
UMBRAL_SENTIDO = 0.70
BATCH_SIZE = 8

print(f"📋 Clasificando {len(df_limpio)} frases por sentido...")

frases_validas = []
frases = list(df_limpio["quote"])

# Clasificación por sentido
for i in tqdm(range(0, len(frases), BATCH_SIZE), desc="Clasificando sentido"):
    batch = frases[i:i+BATCH_SIZE]
    try:
        resultados = sentido_classifier(batch, candidate_labels=labels_sentido)
        if isinstance(resultados, dict):
            resultados = [resultados]
        for frase, resultado in zip(batch, resultados):
            if resultado["labels"][0] == "meaningful" and resultado["scores"][0] >= UMBRAL_SENTIDO:
                frases_validas.append({
                    "quote": frase,
                    "sentido_score": resultado["scores"][0]
                })
    except Exception as e:
        print(f"⚠️ Error en batch {i}: {e}")

print(f"✅ Frases con sentido aceptadas: {len(frases_validas)}")


📋 Clasificando 4197 frases por sentido...


Clasificando sentido: 100%|██████████| 525/525 [00:58<00:00,  9.00it/s]

✅ Frases con sentido aceptadas: 2488


### Clasificación de Frases por Tema

Tras filtrar las frases con sentido, se clasifica el tema de cada una utilizando un modelo `zero-shot` con etiquetas predefinidas como: `"leadership"`, `"motivation"`, `"success"`, `"failure"` y `"risk"`.

Se define un umbral (`UMBRAL_TEMA = 0.70`) para considerar una clasificación de tema como confiable. Solo las frases con una puntuación igual o superior a este umbral son incluidas en el resultado final.

El proceso se realiza por lotes para optimizar el rendimiento y las frases clasificadas se almacenan con sus respectivos scores de sentido y tema. El resultado se guarda en un DataFrame llamado `df_filtradas`.

Finalmente, se liberan los recursos de la GPU para evitar problemas de memoria.


In [21]:
UMBRAL_TEMA = 0.70

print(f"\n🎯 Clasificando tema para {len(frases_validas)} frases...")

frases_filtradas = []
frases_texto = [f["quote"] for f in frases_validas]

for i in tqdm(range(0, len(frases_texto), BATCH_SIZE), desc="Clasificando tema"):
    batch = frases_texto[i:i+BATCH_SIZE]
    try:
        resultados = tema_classifier(batch, candidate_labels=labels_temas)
        if isinstance(resultados, dict):
            resultados = [resultados]
        for f, tema in zip(frases_validas[i:i+BATCH_SIZE], resultados):
            frases_filtradas.append({
                "quote": f["quote"],
                "tema": tema["labels"][0],
                "tema_score": tema["scores"][0],
                "sentido_score": f["sentido_score"]
            })
    except Exception as e:
        print(f"⚠️ Error en batch {i}: {e}")

# Crear DataFrame final
df_filtradas = pd.DataFrame(frases_filtradas)
print(f"📄 Total final de frases clasificadas: {len(df_filtradas)}")

# Liberar memoria GPU
torch.cuda.empty_cache()
gc.collect()


🎯 Clasificando tema para 2488 frases...


Clasificando tema: 100%|██████████| 311/311 [01:20<00:00,  3.87it/s]


📄 Total final de frases clasificadas: 2488


351

### Reclasificación de Frases con Baja Confianza en el Tema

Una vez clasificadas las frases por tema, se realiza una segunda validación para mejorar la precisión:

1. **Separación inicial**:  
   Las frases con una puntuación de tema (`tema_score`) inferior al umbral (`UMBRAL_TEMA = 0.70`) se separan para reclasificación. Las que superan el umbral se consideran válidas directamente.

2. **Reclasificación en lotes**:  
   Las frases con baja confianza son reenviadas al modelo `zero-shot` para una segunda clasificación. Se obtienen nuevos valores para `tema_corregido` y `tema_score_corregido`.

3. **Consolidación**:  
   Todas las frases (válidas inicialmente y reclasificadas) se unifican en un nuevo DataFrame final (`df_final_validado`), listo para análisis o generación.

Finalmente, se liberan los recursos de GPU para evitar sobrecarga de memoria.


In [22]:
UMBRAL_TEMA = 0.70
BATCH_SIZE = 8

frases_validadas = []
frases_a_reclasificar = []
frases_resto = []

# 1. Separar frases según tema_score
for fila in frases_filtradas:
    if fila["tema_score"] < UMBRAL_TEMA:
        frases_a_reclasificar.append(fila)
    else:
        fila["tema_corregido"] = fila["tema"]
        fila["tema_score_corregido"] = fila["tema_score"]
        frases_validadas.append(fila)

print(f"🔍 Reclasificando {len(frases_a_reclasificar)} frases con baja confianza de tema...")

# 2. Reclasificación por lotes
for i in tqdm(range(0, len(frases_a_reclasificar), BATCH_SIZE), desc="Reclasificando tema"):
    batch_filas = frases_a_reclasificar[i:i+BATCH_SIZE]
    batch_texto = [f["quote"] for f in batch_filas]

    try:
        resultados = tema_classifier(batch_texto, candidate_labels=labels_temas)
        if isinstance(resultados, dict):
            resultados = [resultados]
        for fila, resultado in zip(batch_filas, resultados):
            fila["tema_corregido"] = resultado["labels"][0]
            fila["tema_score_corregido"] = resultado["scores"][0]
            frases_validadas.append(fila)
    except Exception as e:
        print(f"⚠️ Error en batch {i}: {e}")

# 3. Convertir a DataFrame final
df_final_validado = pd.DataFrame(frases_validadas)
print(f"✅ Total frases validadas y corregidas: {len(df_final_validado)}")

import gc
import torch

torch.cuda.empty_cache()
gc.collect()


🔍 Reclasificando 1814 frases con baja confianza de tema...


Reclasificando tema: 100%|██████████| 227/227 [00:59<00:00,  3.84it/s]


✅ Total frases validadas y corregidas: 2488


0

### Guardado de Resultados Finales

Después del proceso de clasificación y reclasificación, se guarda el DataFrame final con las frases validadas y sus respectivos temas corregidos.

- El archivo generado se llama: `frases_validadas_con_tema.csv`.
- Contiene las siguientes columnas: la frase original (`quote`), su clasificación temática corregida (`tema_corregido`), los puntajes de confianza (`sentido_score`, `tema_score_corregido`), entre otras.

Este archivo representa el conjunto final de citas inspiradoras clasificadas por tema y listas para su uso.


In [23]:
# Guardar DataFrame validado con temas corregidos
df_final_validado = pd.DataFrame(frases_validadas)
df_final_validado.to_csv("frases_validadas_con_tema.csv", index=False)
print("✅ Archivo guardado como frases_validadas_con_tema.csv")


✅ Archivo guardado como frases_validadas_con_tema.csv


### Carga y Validación del Archivo de Frases Validadas

Se carga el archivo `frases_validadas_con_tema.csv`, que contiene las frases generadas junto con sus temas corregidos y puntajes de validación.

Antes de continuar con el procesamiento, se verifica que las columnas clave existan:

- `"quote"`: contiene la frase inspiradora generada.
- `"tema_corregido"`: contiene el tema final asignado a la frase (por ejemplo: motivation, leadership, etc.).

Esto garantiza que el archivo está completo y estructurado correctamente antes de pasar a la siguiente etapa del proyecto.


In [24]:
# Asegúrate de tener este archivo subido o disponible
df_frases = pd.read_csv("frases_validadas_con_tema.csv")

# Validación de columnas
assert "quote" in df_frases.columns
assert "tema_corregido" in df_frases.columns

### Generación de Nuevas Frases por Tema

Este bloque de código recorre cada uno de los temas definidos (`labels_temas`) y genera nuevas frases inspiradoras basadas en ejemplos reales previamente validados.

#### Proceso:

1. **Filtrado de frases por tema**:
   - Se seleccionan frases ya clasificadas y corregidas que pertenecen al tema actual.

2. **Verificación de cantidad**:
   - Si hay menos de 5 frases disponibles para un tema, se omite para evitar generación poco confiable.

3. **Construcción del prompt**:
   - Se eligen hasta 10 frases reales como inspiración.
   - Se construye un prompt en inglés pidiendo una cita original, profunda y emocional, evitando repeticiones y nombres de autores.

4. **Generación con modelo de lenguaje**:
   - Se tokeniza el prompt y se genera texto usando el modelo de lenguaje previamente cargado (`Mixtral` o similar).
   - Se generan 3 frases por tema con sampling y temperatura controlada.

5. **Postprocesamiento**:
   - Se elimina el prompt de la salida generada.
   - Cada frase generada se guarda junto al tema y el prompt utilizado.

Este proceso permite producir frases nuevas que se alinean con el estilo y contenido típico de citas inspiradoras sobre temas como liderazgo, motivación, éxito, fracaso y riesgo.


In [ ]:
import torch

frases_generadas = []

for tema in tqdm(labels_temas, desc="🔁 Generando por tema"):
    frases_tema = df_frases[df_frases["tema_corregido"] == tema]["quote"].dropna()
    disponibles = len(frases_tema)

    print(f"\n📝 Generando frases para tema: {tema} (disponibles: {disponibles})")

    if disponibles < 5:
        print(f"⚠️ Muy pocas frases para el tema: {tema}, se salta.")
        continue

    ejemplos = frases_tema.sample(n=min(10, disponibles), random_state=42).apply(
        lambda x: x.strip()[:120] + "..." if len(x) > 120 else x.strip()
    ).tolist()

    prompt = (
        f"You are a masterful writer of motivational quotes.\n"
        f"Write an original, deep and emotionally impactful quote about **{tema}**.\n"
        f"Do not include any author names. Avoid clichés and repetition.\n"
        f"Make sure the quote feels authentic and timeless.\n"
        f"Use the following quotes only as inspiration:\n\n"
        + "\n".join(f"- {ej}" for ej in ejemplos) + "\n\n-"
    )

    try:
        print("⏳ Generando...")

        # Tokenizar entrada
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

        # Generar con modelo directamente
        output_ids = model.generate(
            input_ids=input_ids,
            max_new_tokens=50,
            do_sample=True,
            temperature=0.85,
            num_return_sequences=3,
            pad_token_id=tokenizer.eos_token_id
        )

        # Decodificar cada secuencia generada
        for output in output_ids:
            decoded = tokenizer.decode(output, skip_special_tokens=True)
            # Eliminar el prompt de la salida
            texto_generado = decoded.replace(prompt, "").strip()
            frases_generadas.append({
                "tema": tema,
                "prompt": prompt,
                "frase_generada": texto_generado
            })

    except Exception as e:
        print(f"❌ Error al generar frases para tema {tema}: {e}")


🔁 Generando por tema:   0%|          | 0/5 [00:00<?, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



📝 Generando frases para tema: leadership (disponibles: 397)
⏳ Generando...


🔁 Generando por tema:  20%|██        | 1/5 [05:47<23:11, 347.84s/it]


📝 Generando frases para tema: motivation (disponibles: 589)
⏳ Generando...


### Guardar Frases Generadas

Una vez completada la generación de nuevas frases inspiradoras para cada tema, se consolidan en un DataFrame llamado `df_generadas`. Este contiene las siguientes columnas:

- `tema`: tema sobre el que trata la frase.
- `prompt`: texto de entrada usado como inspiración para la generación.
- `frase_generada`: la cita generada por el modelo.

El DataFrame se guarda en un archivo CSV llamado **`nuevas_frases_por_tema.csv`**, sin incluir el índice de pandas, para facilitar su uso posterior.

Además, se imprime un mensaje de confirmación indicando el total de frases generadas exitosamente.


In [ ]:
df_generadas = pd.DataFrame(frases_generadas)
df_generadas.to_csv("nuevas_frases_por_tema.csv", index=False)
print(f"✅ Generación completada. Total frases generadas: {len(df_generadas)}")


### Validación de Frases Generadas

Después de generar nuevas frases motivacionales, se procede a validar su calidad utilizando dos criterios:

1. **Sentido**: Se verifica si la frase es considerada "meaningful" (con sentido) utilizando el clasificador de sentido. Se acepta solo si el `sentido_score ≥ 0.70`.
2. **Tema**: Si la frase tiene sentido, se clasifica temáticamente. Se acepta únicamente si el `tema_score ≥ 0.70`.

#### Proceso:

- Las frases se procesan una por una usando `tqdm` para mostrar progreso.
- Las frases válidas se almacenan en `frases_validadas`, incluyendo información como el tema original, el tema predicho, los puntajes de sentido y tema, y el prompt usado.
- Las frases que no cumplen con los umbrales son descartadas y registradas en `frases_descartadas` junto con el motivo.

#### Resultados:

- Las frases válidas se guardan en **`frases_generadas_validadas.csv`**.
- Las frases descartadas se guardan en **`frases_generadas_descartadas.csv`**.

Se imprime un resumen final con la cantidad de frases aceptadas y rechazadas.


In [ ]:
from tqdm import tqdm

frases_validadas = []
frases_descartadas = []

if 'df_generadas' not in globals():
    df_generadas = pd.read_csv("nuevas_frases_por_tema.csv")

print(f"🔍 Validando {len(df_generadas)} frases generadas...\n")

for i, row in tqdm(df_generadas.iterrows(), total=len(df_generadas), desc="✔️ Verificando"):
    frase = row["frase_generada"]
    tema_original = row["tema"]
    prompt = row["prompt"]

    try:
        # Verificar si tiene sentido
        resultado_sentido = sentido_classifier(frase, candidate_labels=labels_sentido)
        sentido_label = resultado_sentido["labels"][0]
        sentido_score = resultado_sentido["scores"][0]

        if sentido_label == "meaningful" and sentido_score >= 0.70:
            # Clasificar tema
            resultado_tema = tema_classifier(frase, candidate_labels=labels_temas)
            tema_predicho = resultado_tema["labels"][0]
            tema_score = resultado_tema["scores"][0]

            if tema_score >= 0.70:
                frases_validadas.append({
                    "tema_original": tema_original,
                    "prompt_usado": prompt,
                    "frase_generada": frase,
                    "tema_predicho": tema_predicho,
                    "tema_score": tema_score,
                    "sentido_score": sentido_score
                })
            else:
                frases_descartadas.append({
                    "tema_original": tema_original,
                    "prompt_usado": prompt,
                    "frase_generada": frase,
                    "motivo": "bajo tema_score",
                    "tema_predicho": tema_predicho,
                    "tema_score": tema_score,
                    "sentido_score": sentido_score
                })
        else:
            frases_descartadas.append({
                "tema_original": tema_original,
                "prompt_usado": prompt,
                "frase_generada": frase,
                "motivo": "no tiene sentido",
                "sentido_label": sentido_label,
                "sentido_score": sentido_score
            })

    except Exception as e:
        frases_descartadas.append({
            "tema_original": tema_original,
            "prompt_usado": prompt,
            "frase_generada": frase,
            "motivo": f"error de clasificación: {str(e)}"
        })

# Guardar resultados
df_validadas = pd.DataFrame(frases_validadas)
df_descartadas = pd.DataFrame(frases_descartadas)

df_validadas.to_csv("frases_generadas_validadas.csv", index=False)
df_descartadas.to_csv("frases_generadas_descartadas.csv", index=False)

print(f"\n✔️ Frases válidas guardadas: {len(df_validadas)}")
print(f"❌ Frases descartadas: {len(df_descartadas)}")


### Análisis de Frases Generadas Validadas

Una vez completado el proceso de generación y validación, se analiza la distribución y calidad de las frases aceptadas.

#### 1. Carga y Exploración Inicial
Se cargan las frases validadas desde `frases_generadas_validadas.csv` y se inspeccionan las primeras filas y las columnas disponibles.

#### 2. Distribución por Tema
Se calcula el número y porcentaje de frases por cada tema (`tema_predicho`) y se presenta un gráfico de barras que visualiza dicha distribución.

#### 3. Frases Destacadas
Para cada tema, se identifica la **frase con mayor score temático**, lo cual permite destacar las citas más representativas y coherentes de cada categoría.

#### 4. Resultados Presentados:
- Tabla con la **distribución de cantidad y porcentaje** de frases por tema.
- **Gráfico de barras** con la cantidad de frases generadas por tema.
- Tabla con **las mejores frases por tema**, incluyendo:
  - Frase generada
  - Score del tema (`tema_score`)
  - Score de sentido (`sentido_score`)

Este análisis permite evaluar el equilibrio temático y la calidad de las frases generadas por el modelo.

> (Opcional) Se puede guardar la tabla de frases destacadas con `df_top.to_csv("frases_destacadas_por_tema.csv")`.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Cargar frases válidas
df_validadas = pd.read_csv("frases_generadas_validadas.csv")

# Mostrar estructura básica
print("📄 Columnas disponibles:", df_validadas.columns.tolist())
print("\n🟢 Ejemplos de frases válidas:")
display(df_validadas.head(5))

# ✅ Conteo por tema
conteo_por_tema = df_validadas["tema_predicho"].value_counts()
porcentaje_por_tema = conteo_por_tema / conteo_por_tema.sum() * 100

print("\n📊 Distribución de frases por tema:")
print(pd.DataFrame({
    "Cantidad": conteo_por_tema,
    "Porcentaje (%)": porcentaje_por_tema.round(2)
}))

# 📈 Gráfico de barras
plt.figure(figsize=(8, 5))
conteo_por_tema.plot(kind='bar', color='cornflowerblue', edgecolor='black')
plt.title("Distribución de frases generadas por tema", fontsize=14)
plt.xlabel("Tema")
plt.ylabel("Cantidad de frases")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 🔝 Frases más fuertes por tema (en DataFrame para mejor formato)
print("\n🌟 Frases destacadas por tema:")
top_frases = []

for tema in df_validadas["tema_predicho"].unique():
    top = df_validadas[df_validadas["tema_predicho"] == tema].sort_values("tema_score", ascending=False).iloc[0]
    top_frases.append({
        "Tema": tema,
        "Frase destacada": top["frase_generada"],
        "Score de Tema": round(top["tema_score"], 3),
        "Score de Sentido": round(top["sentido_score"], 3)
    })

df_top = pd.DataFrame(top_frases)
display(df_top)

# (Opcional) Guardar resumen
# df_top.to_csv("frases_destacadas_por_tema.csv", index=False)


## ✅ Conclusión del Proyecto

Este proyecto logró desarrollar un sistema automatizado capaz de generar, clasificar y validar frases motivacionales originales utilizando modelos de lenguaje de última generación. Se integraron modelos generativos como Mixtral y clasificadores semánticos DeBERTa para asegurar tanto la calidad del contenido como su alineación temática. El proceso incluyó fases rigurosas de limpieza, filtrado por sentido, clasificación temática y validación cruzada, lo cual permitió construir un conjunto confiable de frases organizadas por temas como liderazgo, motivación, éxito, fracaso y riesgo.

Los resultados demuestran que es posible combinar generación creativa con control semántico para producir contenido original de alto valor, eliminando frases incoherentes o genéricas a través de una validación automática efectiva.

---

## 🔜 Próximos Pasos

Aunque este proyecto culmina dentro del marco de la asignatura, se han identificado varias oportunidades para continuar su desarrollo:

1. **Ampliación del dataset de entrenamiento**: Incluir frases de más fuentes y aumentar la diversidad temática con nuevos tópicos como resiliencia, propósito o perseverancia.

2. **Entrenamiento fino (fine-tuning)**: Afinar modelos como Mistral o LLaMA con un corpus especializado en frases inspiradoras y filosóficas.

3. **Despliegue en una aplicación web**: Crear una plataforma donde los usuarios puedan consultar, filtrar o generar frases según su estado de ánimo o propósito.

4. **Sistema de retroalimentación humana**: Integrar validación por usuarios reales para ajustar los clasificadores con datos de preferencia reales.

5. **Análisis multilingüe**: Extender el sistema a otros idiomas (como español o francés) para ampliar su alcance global.

6. **Integración con redes sociales o APIs**: Automatizar la publicación o distribución de frases validadas en plataformas como Twitter o Instagram.

Este proyecto no solo sirvió como ejercicio técnico en NLP y generación de texto, sino también como punto de partida para aplicaciones más amplias en creatividad asistida por inteligencia artificial. Su potencial continúa más allá de la materia, con posibilidades reales de impacto en productos digitales, educación, motivación personal y más.
